In [7]:
import numpy as np
import sklearn
from sentence_transformers import SentenceTransformer
import torch
from transformers import CLIPTokenizer, CLIPTextModel, T5Tokenizer, T5EncoderModel

In [2]:
sub1 = np.load("/ssd_scratch/cvit/souvik/subj1.npy", allow_pickle=True).item()
sub2 = np.load("/ssd_scratch/cvit/souvik/subj2.npy", allow_pickle=True).item()

In [3]:
sub1.keys()

dict_keys(['language', 'vision', 'dmn', 'task'])

In [4]:
sub1['language'].shape, sub1['vision'].shape, sub1['dmn'].shape, sub1['task'].shape



((627, 11437), (627, 33792), (627, 17190), (627, 35120))

In [9]:
with open('/ssd_scratch/cvit/souvik/stimuli.txt', 'r') as file:
    sentences = [line.strip() for line in file if line.strip()]

In [8]:
model_large = SentenceTransformer("all-mpnet-base-v2")
model_small = SentenceTransformer("all-MiniLM-L6-v2")
model_name = "openai/clip-vit-base-patch32"
tokenizer = CLIPTokenizer.from_pretrained(model_name)
model_clip = CLIPTextModel.from_pretrained(model_name)
model_name_t5 = "t5-base"  # You can use different sizes like t5-small or t5-large
tokenizer_t5 = T5Tokenizer.from_pretrained(model_name_t5)
model_t5 = T5EncoderModel.from_pretrained(model_name_t5)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not inst

In [11]:
embeddings_large = model_large.encode(sentences)
embeddings_small = model_small.encode(sentences)


In [ ]:
embeddings_clip = []
for sentence in sentences:
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model_clip(**inputs)
    
    # Get the [CLS] token embedding which represents the whole sentence
    embedding = outputs.last_hidden_state[:, 0, :].numpy()
    embeddings_clip.append(embedding)


embeddings_t5 = []
for sentence in sentences:
    inputs = tokenizer_t5(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model_t5(**inputs)
    
    # Get sentence embedding by averaging token embeddings
    # Alternatively, you could use the last token or other pooling strategies
    mask = inputs.attention_mask.unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
    masked_embeddings = outputs.last_hidden_state * mask
    summed = torch.sum(masked_embeddings, dim=1)
    counts = torch.clamp(torch.sum(mask, dim=1), min=1e-9)
    mean_pooled = summed / counts
    embedding = mean_pooled.numpy()
    embeddings_t5.append(embedding)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


AttributeError: 'list' object has no attribute 'shape'

In [16]:
embeddings_clip = np.array(embeddings_clip).squeeze(1)
embeddings_t5 = np.array(embeddings_t5).squeeze(1)  
embeddings_large.shape, embeddings_small.shape, embeddings_clip.shape, embeddings_t5.shape

((627, 768), (627, 384), (627, 512), (627, 768))

In [18]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine

def compute_2v2_accuracy(Y_true, Y_pred):
    """
    Compute 2v2 accuracy between true and predicted voxel activations.
    
    Args:
        Y_true: True voxel activations with shape (N, V) where N is number of samples and V is number of voxels
        Y_pred: Predicted voxel activations with shape (N, V)
        
    Returns:
        2v2 accuracy score (float)
    """
    N = Y_true.shape[0]
    count_correct = 0
    
    # Compute all pairwise comparisons
    total_pairs = 0
    for i in range(N-1):
        for j in range(i+1, N):
            # Compute cosine distances
            cos_dist_i_i_hat = cosine(Y_true[i], Y_pred[i])
            cos_dist_j_j_hat = cosine(Y_true[j], Y_pred[j])
            cos_dist_i_j_hat = cosine(Y_true[i], Y_pred[j])
            cos_dist_j_i_hat = cosine(Y_true[j], Y_pred[i])
            
            # 2v2 condition: sum of correct pairings < sum of incorrect pairings
            if (cos_dist_i_i_hat + cos_dist_j_j_hat) < (cos_dist_i_j_hat + cos_dist_j_i_hat):
                count_correct += 1
            
            total_pairs += 1
    
    return count_correct / total_pairs if total_pairs > 0 else 0

def compute_pearson_correlation(Y_true, Y_pred):
    """
    Compute average Pearson correlation between true and predicted voxel activations.
    
    Args:
        Y_true: True voxel activations with shape (N, V)
        Y_pred: Predicted voxel activations with shape (N, V)
        
    Returns:
        Average Pearson correlation across all samples
    """
    N = Y_true.shape[0]
    correlations = []
    
    for i in range(N):
        corr, _ = pearsonr(Y_true[i], Y_pred[i])
        correlations.append(corr)
    
    return np.mean(correlations)

def cross_validate_ridge_regression(embeddings, voxels, n_folds=5, alphas=None):
    """
    Perform k-fold cross-validation for ridge regression from embeddings to voxels.
    
    Args:
        embeddings: Sentence embeddings with shape (627, 384)
        voxels: Voxel activations with shape (627, 10791)
        n_folds: Number of folds for cross-validation
        alphas: List of alpha values to try for ridge regression
        
    Returns:
        Dictionary with average metrics across folds for each alpha
    """
    if alphas is None:
        alphas = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    results = {alpha: {'2v2_acc': [], 'pearson_corr': []} for alpha in alphas}
    
    for train_idx, test_idx in kf.split(embeddings):
        # Split data into train and test sets for this fold
        X_train, X_test = embeddings[train_idx], embeddings[test_idx]
        y_train, y_test = voxels[train_idx], voxels[test_idx]
        
        for alpha in alphas:
            # Train Ridge regression model
            model = Ridge(alpha=alpha)
            model.fit(X_train, y_train)
            
            # Make predictions
            y_pred = model.predict(X_test)
            
            # Compute metrics
            acc_2v2 = compute_2v2_accuracy(y_test, y_pred)
            pearson_corr = compute_pearson_correlation(y_test, y_pred)
            
            # Store results for this fold
            results[alpha]['2v2_acc'].append(acc_2v2)
            results[alpha]['pearson_corr'].append(pearson_corr)
    
    # Compute average metrics across folds
    avg_results = {}
    for alpha in alphas:
        avg_results[alpha] = {
            '2v2_acc': np.mean(results[alpha]['2v2_acc']),
            'pearson_corr': np.mean(results[alpha]['pearson_corr'])
        }
    
    return avg_results, results, model

def find_best_alpha(avg_results):
    """Find the alpha with the best performance based on both metrics"""
    # Normalize both metrics to [0, 1] for fair comparison
    acc_values = np.array([avg_results[alpha]['2v2_acc'] for alpha in avg_results])
    corr_values = np.array([avg_results[alpha]['pearson_corr'] for alpha in avg_results])
    
    # Normalize (min-max scaling)
    acc_norm = (acc_values - np.min(acc_values)) / (np.max(acc_values) - np.min(acc_values))
    corr_norm = (corr_values - np.min(corr_values)) / (np.max(corr_values) - np.min(corr_values))
    
    # Compute combined score (equal weighting)
    combined_scores = 0.5 * acc_norm + 0.5 * corr_norm
    
    # Get best alpha
    alphas = list(avg_results.keys())
    best_idx = np.argmax(combined_scores)
    best_alpha = alphas[best_idx]
    
    return best_alpha

## Encoding

In [24]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge = cross_validate_ridge_regression(
    embeddings_clip, sub1['language'], n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.0226		0.7535


In [20]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge = cross_validate_ridge_regression(
    embeddings_clip, sub1['vision'], n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.0416		0.7939


In [39]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge_lang = cross_validate_ridge_regression(
    embeddings_small, sub1['language'], n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.7767		0.6094


In [40]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge_vision = cross_validate_ridge_regression(
    embeddings_small, sub1['vision'], n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.7831		0.6751


In [41]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge_task = cross_validate_ridge_regression(
    embeddings_small, sub1['task'], n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.7120		0.4054


In [42]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge_dmn = cross_validate_ridge_regression(
    embeddings_small, sub1['dmn'], n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.6872		0.3926


In [28]:
import spacy

nlp = spacy.load("en_core_web_sm")

def mask_by_pos(text, pos_to_keep):
    doc = nlp(text)
    return ' '.join([token.text if token.pos_ in pos_to_keep else '' for token in doc])

In [33]:
sentence = sentences[-10]
masked_nouns = mask_by_pos(sentence, {"NOUN"})
masked_verbs = mask_by_pos(sentence, {"VERB"})
masked_adjs = mask_by_pos(sentence, {"ADJ"})
print(sentence, "|",masked_nouns, "|",masked_verbs, "|",masked_adjs)

I hesitantly skied down the steep trail that my buddies convinced me to try. |       trail   buddies      |   skied        convinced   try  |      steep         


In [34]:
def get_single_embedding(text):
    emb_big = model_large.encode([text])
    emb_small = model_small.encode([text])
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model_clip(**inputs)
    emb_clip = outputs.last_hidden_state[:, 0, :].numpy()
    return emb_big, emb_small, emb_clip

In [38]:
embedding_test_full = embeddings_small[-10]
embedding_test_nouns = get_single_embedding(masked_nouns)[1]
embedding_test_verbs = get_single_embedding(masked_verbs)[1]
embedding_test_adjs = get_single_embedding(masked_adjs)[1]
print(embedding_test_full.shape, embedding_test_nouns.shape, embedding_test_verbs.shape, embedding_test_adjs.shape)


(384,) (1, 384) (1, 384) (1, 384)


In [46]:
fmri_list[0][-10].shape

(11437,)

In [52]:
# lets test for nouns first
model_lists = [model_ridge_lang, model_ridge_vision, model_ridge_task, model_ridge_dmn]
model_names = ["language", "vision", "task", "dmn"]
fmri_list = [sub1['language'], sub1['vision'], sub1['task'], sub1['dmn']]
for i, model in enumerate(model_lists):
    pred_fmri_noun = model.predict(embedding_test_nouns)
    pred_fmri_noun = pred_fmri_noun.squeeze(0)
    corr,_ = pearsonr(fmri_list[i][-10], pred_fmri_noun)
    print(f"Correlation with {model_names[i]} for nouns: {corr:.4f}")



Correlation with language for nouns: 0.5355
Correlation with vision for nouns: 0.5889
Correlation with task for nouns: 0.4942
Correlation with dmn for nouns: 0.4902


In [53]:
# verbs
model_lists = [model_ridge_lang, model_ridge_vision, model_ridge_task, model_ridge_dmn]
model_names = ["language", "vision", "task", "dmn"]
fmri_list = [sub1['language'], sub1['vision'], sub1['task'], sub1['dmn']]
for i, model in enumerate(model_lists):
    pred_fmri_verbs = model.predict(embedding_test_verbs)
    pred_fmri_verbs = pred_fmri_verbs.squeeze(0)
    corr,_ = pearsonr(fmri_list[i][-10], pred_fmri_verbs)
    print(f"Correlation with {model_names[i]} for verbs: {corr:.4f}")


Correlation with language for verbs: 0.5720
Correlation with vision for verbs: 0.7075
Correlation with task for verbs: 0.2898
Correlation with dmn for verbs: 0.2352


In [54]:
# adjs
model_lists = [model_ridge_lang, model_ridge_vision, model_ridge_task, model_ridge_dmn]
model_names = ["language", "vision", "task", "dmn"]
fmri_list = [sub1['language'], sub1['vision'], sub1['task'], sub1['dmn']]
for i, model in enumerate(model_lists):
    pred_fmri_adjs = model.predict(embedding_test_adjs)
    pred_fmri_adjs = pred_fmri_adjs.squeeze(0)
    corr,_ = pearsonr(fmri_list[i][-10], pred_fmri_adjs)
    print(f"Correlation with {model_names[i]} for verbs: {corr:.4f}")

Correlation with language for verbs: 0.2614
Correlation with vision for verbs: 0.5526
Correlation with task for verbs: 0.1297
Correlation with dmn for verbs: -0.0942


## Decoding

In [22]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge = cross_validate_ridge_regression(
    sub1['language'],embeddings_small, n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.9958		0.3266


In [56]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge = cross_validate_ridge_regression(
    sub1['language'],embeddings_large, n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.9967		0.3487


In [63]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge = cross_validate_ridge_regression(
    sub1['language'],embeddings_clip, n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.0000		1.0000


In [64]:
alphas = [0.01]
    
# Perform cross-validation
avg_results, detailed_results, model_ridge = cross_validate_ridge_regression(
    sub1['language'],embeddings_t5, n_folds=5, alphas=alphas
)

# Print results for each alpha
print("\nCross-validation results:")
print("Alpha\t2V2 Accuracy\tPearson Correlation")
print("-" * 50)
for alpha in alphas:
    print(f"{alpha}\t{avg_results[alpha]['2v2_acc']:.4f}\t\t{avg_results[alpha]['pearson_corr']:.4f}")


Cross-validation results:
Alpha	2V2 Accuracy	Pearson Correlation
--------------------------------------------------
0.01	0.9958		0.6499


In [65]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
import time
import numpy as np
from joblib import Parallel, delayed

class BrainRegionDecoder:
    """
    Decoder class for a single brain region
    """
    def __init__(self, alpha=1.0):
        self.model = Ridge(alpha=alpha)
        self.alpha = alpha
        
    def fit(self, voxels, embeddings):
        """Train the model to predict embeddings from voxels"""
        self.model.fit(voxels, embeddings)
        return self
        
    def predict(self, voxels):
        """Predict embeddings from voxels"""
        return self.model.predict(voxels)

class StackedBrainDecoder:
    """
    Stacked regressor that combines predictions from multiple brain regions
    """
    def __init__(self, alphas=None, meta_alpha=1.0, n_folds=5):
        """
        Initialize the stacked decoder
        
        Args:
            alphas: Dictionary mapping region names to alpha values for Ridge regression
            meta_alpha: Alpha value for the meta-regressor
            n_folds: Number of folds for cross-validation during level-1 training
        """
        self.region_decoders = {}
        self.meta_regressor = Ridge(alpha=meta_alpha)
        self.alphas = alphas if alphas is not None else {}
        self.n_folds = n_folds
        self.meta_alpha = meta_alpha
        
    def fit(self, region_voxels, embeddings):
        """
        Train the stacked model
        
        Args:
            region_voxels: Dictionary mapping region names to voxel arrays
            embeddings: Target embedding array of shape (n_samples, embedding_dim)
        """
        print("Training first-level decoders...")
        n_samples = embeddings.shape[0]
        
        # Generate out-of-fold predictions for each region
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        
        # Create empty arrays to hold level-1 training data
        region_predictions = {}
        for region_name in region_voxels:
            region_predictions[region_name] = np.zeros_like(embeddings)
            
        # Generate out-of-fold predictions
        for region_name, voxels in region_voxels.items():
            print(f"  Processing region: {region_name}")
            alpha = self.alphas.get(region_name, 1.0)
            
            for train_idx, val_idx in kf.split(voxels):
                # Create and train the decoder for this fold
                decoder = BrainRegionDecoder(alpha=alpha)
                decoder.fit(voxels[train_idx], embeddings[train_idx])
                
                # Make predictions for validation fold
                region_predictions[region_name][val_idx] = decoder.predict(voxels[val_idx])
                
            # Train the final decoder on all data
            self.region_decoders[region_name] = BrainRegionDecoder(alpha=alpha).fit(voxels, embeddings)
        
        # Combine predictions from all regions for meta-training
        print("Training meta-regressor...")
        meta_features = np.hstack([region_predictions[region] for region in region_voxels])
        self.meta_regressor.fit(meta_features, embeddings)
        
        return self
    
    def predict(self, region_voxels):
        """
        Make predictions using the stacked model
        
        Args:
            region_voxels: Dictionary mapping region names to voxel arrays for test samples
            
        Returns:
            Predicted embeddings
        """
        # Make predictions from each region decoder
        region_predictions = []
        for region_name, voxels in region_voxels.items():
            if region_name in self.region_decoders:
                region_pred = self.region_decoders[region_name].predict(voxels)
                region_predictions.append(region_pred)
        
        # Combine predictions for meta-regressor
        meta_features = np.hstack(region_predictions)
        
        # Make final predictions
        return self.meta_regressor.predict(meta_features)

def compute_2v2_accuracy(Y_true, Y_pred):
    """
    Compute 2v2 accuracy between true and predicted embeddings
    """
    N = Y_true.shape[0]
    count_correct = 0
    total_pairs = 0
    
    for i in range(N-1):
        for j in range(i+1, N):
            # Compute cosine distances
            cos_dist_i_i_hat = cosine(Y_true[i], Y_pred[i])
            cos_dist_j_j_hat = cosine(Y_true[j], Y_pred[j])
            cos_dist_i_j_hat = cosine(Y_true[i], Y_pred[j])
            cos_dist_j_i_hat = cosine(Y_true[j], Y_pred[i])
            
            # 2v2 condition: sum of correct pairings < sum of incorrect pairings
            if (cos_dist_i_i_hat + cos_dist_j_j_hat) < (cos_dist_i_j_hat + cos_dist_j_i_hat):
                count_correct += 1
            
            total_pairs += 1
    
    return count_correct / total_pairs

def compute_pearson_correlation(Y_true, Y_pred):
    """
    Compute average Pearson correlation between true and predicted embeddings
    """
    N = Y_true.shape[0]
    correlations = []
    
    for i in range(N):
        corr, _ = pearsonr(Y_true[i], Y_pred[i])
        correlations.append(corr)
    
    return np.mean(correlations)

def compute_rank_accuracy(Y_true, Y_pred):
    """
    Compute median rank of true sentences based on cosine similarity
    
    For each sample i, we compute the cosine similarity between predicted embedding Y_pred[i]
    and all true embeddings Y_true. We then rank the true sentence among all candidates.
    """
    N = Y_true.shape[0]
    ranks = []
    
    for i in range(N):
        # Compute cosine similarity between prediction i and all true embeddings
        similarities = [1 - cosine(Y_pred[i], Y_true[j]) for j in range(N)]
        
        # Sort similarities (descending) and get rank of true sentence
        sorted_indices = np.argsort(similarities)[::-1]
        rank = np.where(sorted_indices == i)[0][0] + 1  # +1 for 1-indexed ranking
        ranks.append(rank)
    
    return np.median(ranks)

def compute_top_k_accuracy(Y_true, Y_pred, k=5):
    """
    Compute percentage of cases where true sentence is in top-k predictions
    """
    N = Y_true.shape[0]
    top_k_correct = 0
    
    for i in range(N):
        # Compute cosine similarity between prediction i and all true embeddings
        similarities = [1 - cosine(Y_pred[i], Y_true[j]) for j in range(N)]
        
        # Sort similarities (descending) and get rank of true sentence
        sorted_indices = np.argsort(similarities)[::-1]
        rank = np.where(sorted_indices == i)[0][0] + 1  # +1 for 1-indexed ranking
        
        if rank <= k:
            top_k_correct += 1
    
    return top_k_correct / N

def evaluate_decoder(Y_true, Y_pred, ks=[1, 5, 10]):
    """
    Evaluate decoder performance with multiple metrics
    """
    results = {
        '2v2_accuracy': compute_2v2_accuracy(Y_true, Y_pred),
        'pearson_correlation': compute_pearson_correlation(Y_true, Y_pred),
        'median_rank': compute_rank_accuracy(Y_true, Y_pred)
    }
    
    # Compute top-k accuracy for different k values
    for k in ks:
        results[f'top_{k}_accuracy'] = compute_top_k_accuracy(Y_true, Y_pred, k=k)
    
    return results

def cross_validate_stacked_decoder(region_voxels, embeddings, n_folds=5, 
                                  region_alphas=None, meta_alpha=1.0):
    """
    Perform cross-validation to evaluate the stacked decoder
    
    Args:
        region_voxels: Dictionary mapping region names to voxel arrays
        embeddings: Target embedding array
        n_folds: Number of folds for cross-validation
        region_alphas: Dictionary mapping region names to alpha values
        meta_alpha: Alpha value for meta-regressor
        
    Returns:
        Dictionary with evaluation metrics
    """
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Store metrics for each fold
    fold_metrics = []
    
    print(f"Starting {n_folds}-fold cross-validation...")
    fold = 1
    
    for train_idx, test_idx in kf.split(embeddings):
        print(f"Fold {fold}/{n_folds}")
        start_time = time.time()
        
        # Prepare training and test data for each region
        train_regions = {}
        test_regions = {}
        
        for region_name, voxels in region_voxels.items():
            train_regions[region_name] = voxels[train_idx]
            test_regions[region_name] = voxels[test_idx]
        
        # Train and evaluate stacked decoder
        stacked_decoder = StackedBrainDecoder(
            alphas=region_alphas,
            meta_alpha=meta_alpha,
            n_folds=3  # Use fewer folds for the internal CV to save time
        )
        
        stacked_decoder.fit(train_regions, embeddings[train_idx])
        predictions = stacked_decoder.predict(test_regions)
        
        # Evaluate performance
        metrics = evaluate_decoder(embeddings[test_idx], predictions)
        fold_metrics.append(metrics)
        
        print(f"  Completed in {time.time() - start_time:.2f} seconds")
        print(f"  2V2 Accuracy: {metrics['2v2_accuracy']:.4f}")
        print(f"  Pearson Correlation: {metrics['pearson_correlation']:.4f}")
        print(f"  Median Rank: {metrics['median_rank']:.1f}")
        print(f"  Top-1 Accuracy: {metrics['top_1_accuracy']:.4f}")
        print(f"  Top-5 Accuracy: {metrics['top_5_accuracy']:.4f}")
        
        fold += 1
    
    # Average metrics across folds
    avg_metrics = {}
    for key in fold_metrics[0].keys():
        avg_metrics[key] = np.mean([fold[key] for fold in fold_metrics])
    
    return avg_metrics, fold_metrics

# Example usage
if __name__ == "__main__":
    # Example data - replace with actual data loading
    n_samples = 627
    embedding_dim = 384
    
    # Create sample region voxels dictionary
    # In real usage, load your actual voxel data for each region
    region_voxels = {
        'language': sub1['language'],
        'vision': sub1['vision']    ,
        'task': sub1['task'],
        'dmn': sub1['dmn']
    }

    # Define region-specific alphas (optimized separately for each region)
    region_alphas = {
        'language': 0.01,
        'vision': 0.01,
        'task': 0.01,
        'dmn': 0.01
    }
    
    # Cross-validate stacked decoder
    avg_metrics, fold_metrics = cross_validate_stacked_decoder(
        region_voxels, 
        embeddings_clip,
        n_folds=5,
        region_alphas=region_alphas,
        meta_alpha=1.0
    )
    
    # Print average metrics
    print("\nAverage metrics across folds:")
    for key, value in avg_metrics.items():
        print(f"{key}: {value:.4f}")

Starting 5-fold cross-validation...
Fold 1/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 11.33 seconds
  2V2 Accuracy: 0.0000
  Pearson Correlation: 1.0000
  Median Rank: 63.5
  Top-1 Accuracy: 0.0079
  Top-5 Accuracy: 0.0397
Fold 2/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 11.28 seconds
  2V2 Accuracy: 0.0000
  Pearson Correlation: 1.0000
  Median Rank: 63.5
  Top-1 Accuracy: 0.0079
  Top-5 Accuracy: 0.0397
Fold 3/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 11.20 seconds
  2V2 Accuracy: 0.0000
  Pearson Correlation: 1.0000
  Median Rank: 63.0
  Top-1 Accuracy: 0.0080

In [66]:

n_samples = 627
embedding_dim = 384

# Create sample region voxels dictionary
# In real usage, load your actual voxel data for each region
region_voxels = {
    'language': sub1['language'],
    'vision': sub1['vision']    ,
    'task': sub1['task'],
    'dmn': sub1['dmn']
}

# Define region-specific alphas (optimized separately for each region)
region_alphas = {
    'language': 0.01,
    'vision': 0.01,
    'task': 0.01,
    'dmn': 0.01
}

# Cross-validate stacked decoder
avg_metrics, fold_metrics = cross_validate_stacked_decoder(
    region_voxels, 
    embeddings_large,
    n_folds=5,
    region_alphas=region_alphas,
    meta_alpha=1.0
)

# Print average metrics
print("\nAverage metrics across folds:")
for key, value in avg_metrics.items():
    print(f"{key}: {value:.4f}")

Starting 5-fold cross-validation...
Fold 1/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 12.40 seconds
  2V2 Accuracy: 0.9967
  Pearson Correlation: 0.3941
  Median Rank: 2.0
  Top-1 Accuracy: 0.4603
  Top-5 Accuracy: 0.7381
Fold 2/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 12.45 seconds
  2V2 Accuracy: 0.9940
  Pearson Correlation: 0.3799
  Median Rank: 2.0
  Top-1 Accuracy: 0.4444
  Top-5 Accuracy: 0.7619
Fold 3/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 12.58 seconds
  2V2 Accuracy: 0.9939
  Pearson Correlation: 0.3714
  Median Rank: 2.0
  Top-1 Accuracy: 0.4640
  

In [68]:
n_samples = 627
embedding_dim = 384

# Create sample region voxels dictionary
# In real usage, load your actual voxel data for each region
region_voxels = {
    'language': sub1['language'],
    'vision': sub1['vision']    ,
    'task': sub1['task'],
    'dmn': sub1['dmn']
}


# Define region-specific alphas (optimized separately for each region)
region_alphas = {
    'language': 0.01,
    'vision': 0.01,
    'task': 0.01,
    'dmn': 0.01
}

# Cross-validate stacked decoder
avg_metrics, fold_metrics = cross_validate_stacked_decoder(
    region_voxels, 
    embeddings_small,
    n_folds=5,
    region_alphas=region_alphas,
    meta_alpha=1.0
)

# Print average metrics
print("\nAverage metrics across folds:")
for key, value in avg_metrics.items():
    print(f"{key}: {value:.4f}")

Starting 5-fold cross-validation...
Fold 1/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 10.35 seconds
  2V2 Accuracy: 0.9940
  Pearson Correlation: 0.3622
  Median Rank: 2.0
  Top-1 Accuracy: 0.4286
  Top-5 Accuracy: 0.7143
Fold 2/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 10.68 seconds
  2V2 Accuracy: 0.9956
  Pearson Correlation: 0.3568
  Median Rank: 2.0
  Top-1 Accuracy: 0.4206
  Top-5 Accuracy: 0.7857
Fold 3/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 10.62 seconds
  2V2 Accuracy: 0.9945
  Pearson Correlation: 0.3418
  Median Rank: 2.0
  Top-1 Accuracy: 0.4080
  

In [69]:
n_samples = 627
embedding_dim = 384

# Create sample region voxels dictionary
# In real usage, load your actual voxel data for each region
region_voxels = {
    'language': sub1['language'],
    'vision': sub1['vision']    ,
    'task': sub1['task'],
    'dmn': sub1['dmn']
}


# Define region-specific alphas (optimized separately for each region)
region_alphas = {
    'language': 0.01,
    'vision': 0.01,
    'task': 0.01,
    'dmn': 0.01
}

# Cross-validate stacked decoder
avg_metrics, fold_metrics = cross_validate_stacked_decoder(
    region_voxels, 
    embeddings_t5,
    n_folds=5,
    region_alphas=region_alphas,
    meta_alpha=1.0
)

# Print average metrics
print("\nAverage metrics across folds:")
for key, value in avg_metrics.items():
    print(f"{key}: {value:.4f}")

Starting 5-fold cross-validation...
Fold 1/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 12.48 seconds
  2V2 Accuracy: 0.9919
  Pearson Correlation: 0.6514
  Median Rank: 3.0
  Top-1 Accuracy: 0.3175
  Top-5 Accuracy: 0.6905
Fold 2/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 12.70 seconds
  2V2 Accuracy: 0.9897
  Pearson Correlation: 0.6485
  Median Rank: 3.0
  Top-1 Accuracy: 0.3333
  Top-5 Accuracy: 0.6349
Fold 3/5
Training first-level decoders...
  Processing region: language
  Processing region: vision
  Processing region: task
  Processing region: dmn
Training meta-regressor...
  Completed in 12.85 seconds
  2V2 Accuracy: 0.9897
  Pearson Correlation: 0.6461
  Median Rank: 4.0
  Top-1 Accuracy: 0.3200
  